In [2]:
from utils import safe_serialize, listar_ops
import pandas as pd

In [ ]:
# Si es notebook, mejor usar:
BASE_DIR = Path.cwd()

# Ir un nivel arriba (de notebooks → raíz del repo)
ROOT = BASE_DIR.parent

# Carpeta data dentro del repo
DATA_DIR = ROOT / "data"
DATA_DIR.mkdir(exist_ok=True)

In [ ]:
listar_ops("https://opendata.camara.cl/camaradiputados/WServices/WSLegislativo.asmx?WSDL")

In [ ]:
c = Client("https://opendata.camara.cl/camaradiputados/WServices/WSLegislativo.asmx?WSDL")
res = c.service.retornarPeriodosLegislativos()
print(res)

In [ ]:
d = safe_serialize(res) or {}
df = pd.json_normalize(d, sep=".")
df

In [ ]:
df_explode = df.explode("Legislaturas.Legislatura").reset_index(drop=True)
df_legislaturas = json_normalize(df_explode["Legislaturas.Legislatura"])
df_legislaturas.head(5)

In [ ]:
df_legislaturas.rename(columns={
    "Id" : "Legislatura.Id",
    "FechaInicio" : "Legislatura.FechaInicio",
    "FechaTermino" : "Legislatura.FechaTermino"
}, inplace=True)

df_legislaturas.head(5)

In [ ]:
df_legislaturas = pd.concat([df_explode, df_legislaturas], axis=1)
df_legislaturas.drop(["Diputados", "Legislaturas.Legislatura", "Legislaturas", 
                      "FechaInicio", "FechaTermino", "Tipo.Valor"], axis=1, inplace=True)
df_legislaturas.rename(columns={"Nombre" : "Periodo"}, inplace=True)
df_legislaturas.head()

Guardar legislaturas

In [ ]:
def sanitize(name):
    """Limpia nombres de carpeta/archivo."""
    if pd.isna(name) or str(name).strip() == "":
        return "sin_periodo"
    return re.sub(r"[^\w\-]+", "_", str(name)).strip("_")

for col in ["Legislatura.FechaInicio", "Legislatura.FechaTermino"]:
    if col in df_legislaturas.columns:
        df_legislaturas[col] = pd.to_datetime(df_legislaturas[col], errors="coerce")

cols = [c for c in [
    "Legislatura.Id", "Numero",
    "Legislatura.FechaInicio", "Legislatura.FechaTermino",
    "Tipo._value_1", "Tipo.Valor"
] if c in df_legislaturas.columns]

# === 1) Split por periodo y guardar ===
for periodo, sub in df_legislaturas.groupby("Periodo", dropna=False):
    carpeta = DATA_DIR / sanitize(periodo)      # p.ej. data/1990-1994
    carpeta.mkdir(parents=True, exist_ok=True)

    # ordenar por fecha y número
    order_cols = [c for c in ["Legislatura.FechaInicio", "Numero"] if c in sub.columns]
    sub_sorted = sub.sort_values(order_cols) if order_cols else sub

    # seleccionar columnas, si no, guarda todo
    to_save = sub_sorted[cols] if cols else sub_sorted

    # guardar
    out_path = carpeta / "legislaturas.csv"
    to_save.to_csv(out_path, index=False, encoding="utf-8")
    print(f"✓ Guardado {out_path}")

In [ ]:
df_periodos = (
    df_legislaturas[["Id", "Periodo"]]
    .drop_duplicates()
    .set_index("Id")
)
df_periodos.to_csv(DATA_DIR / 'periodos.csv', index=False)